In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import sqlite3
import xml.etree.ElementTree as ET
from datetime import datetime
import ipywidgets as widgets
import math

In [3]:
#Lets set up the connection to database
database = "database.sqlite"
conn = sqlite3.connect(database)

In [4]:
#Lets print out the name of the tables in the database
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [5]:
pd.read_sql("SELECT * FROM League;", conn)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [6]:
#Given that we want to find out infomration of the English Premier Leage, lets get the data using SQL query
epl = pd.read_sql("""SELECT id, league_id, stage, season,date, match_api_id,
                     home_team_api_id, (SELECT team_long_name FROM Team WHERE team_api_id = home_team_api_id) home_team, 
                     away_team_api_id, (SELECT team_long_name FROM Team WHERE team_api_id = away_team_api_id) away_team,
                     home_team_goal, away_team_goal, goal, shoton, shotoff, foulcommit,
                     card, cross, corner, possession, B365H, B365D, B365A
                     FROM Match m
                     WHERE league_id = (SELECT id FROM league WHERE name = 'England Premier League')
                     ORDER BY date;
                """, conn)


In [7]:
epl

,id,league_id,stage,season,date,match_api_id,home_team_api_id,home_team,away_team_api_id,away_team,...,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A
0,1730,1729,1,2008/2009,2008-08-16 00:00:00,489043,9825,Arsenal,8659,West Bromwich Albion,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...,1.20,6.50,15.00
1,1731,1729,1,2008/2009,2008-08-16 00:00:00,489044,8472,Sunderland,8650,Liverpool,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,5.50,3.60,1.67
2,1732,1729,1,2008/2009,2008-08-16 00:00:00,489045,8654,West Ham United,8528,Wigan Athletic,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...,1.91,3.40,4.20
3,1734,1729,1,2008/2009,2008-08-16 00:00:00,489047,8668,Everton,8655,Blackburn Rovers,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><event...,2.00,3.30,4.00
4,1735,1729,1,2008/2009,2008-08-16 00:00:00,489048,8549,Middlesbrough,8586,Tottenham Hotspur,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>49</comment><event...,3.20,3.40,2.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,4705,1729,38,2015/2016,2016-05-15 00:00:00,1987603,10194,Stoke City,8654,West Ham United,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>20</comment><stats...,3.50,3.60,2.15
3036,4706,1729,38,2015/2016,2016-05-15 00:00:00,1987604,10003,Swansea City,8456,Manchester City,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>83</comment><stats...,6.00,4.75,1.53
3037,4707,1729,38,2015/2016,2016-05-15 00:00:00,1987605,9817,Watford,8472,Sunderland,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>39</comment><stats...,2.05,3.75,3.70
3038,4708,1729,38,2015/2016,2016-05-15 00:00:00,1987606,8659,West Bromwich Albion,8650,Liverpool,...,<shot

In [8]:
match_count = epl['id'].size
no_seasons = epl['season'].nunique()
print("The dataframe consists of %d rows spanning %d years of data" % (match_count, no_seasons))
epl['season'].value_counts()

The dataframe consists of 3040 rows spanning 8 years of data


2008/2009    380
2009/2010    380
2010/2011    380
2011/2012    380
2012/2013    380
2013/2014    380
2014/2015    380
2015/2016    380
Name: season, dtype: int64

In [9]:
epl.head()

,id,league_id,stage,season,date,match_api_id,home_team_api_id,home_team,away_team_api_id,away_team,...,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A
0,1730,1729,1,2008/2009,2008-08-16 00:00:00,489043,9825,Arsenal,8659,West Bromwich Albion,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...,1.20,6.5,15.00
1,1731,1729,1,2008/2009,2008-08-16 00:00:00,489044,8472,Sunderland,8650,Liverpool,...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,5.50,3.6,1.67
2,1732,1729,1,2008/2009,2008-08-16 00:00:00,489045,8654,West Ham United,8528,Wigan Athletic,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...,1.91,3.4,4.20
3,1734,1729,1,2008/2009,2008-08-16 00:00:00,489047,8668,Everton,8655,Blackburn Rovers,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>46</comment><event...,2.00,3.3,4.00
4,1735,1729,1,2008/2009,2008-08-16 00:00:00,489048,8549,Middlesbrough,8586,Tottenham Hotspur,...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>49</comment><event...,3.20,3.4,2.25


In [10]:
epl['possession']

0       <possession><value><comment>65</comment><event...
1       <possession><value><comment>45</comment><event...
2       <possession><value><comment>50</comment><event...
3       <possession><value><comment>46</comment><event...
4       <possession><value><comment>49</comment><event...
                              ...                        
3035    <possession><value><comment>20</comment><stats...
3036    <possession><value><comment>83</comment><stats...
3037    <possession><value><comment>39</comment><stats...
3038    <possession><value><comment>23</comment><stats...
3039    <possession><value><comment>75</comment><stats...
Name: possession, Length: 3040, dtype: object

In [11]:
def calculate_stats_both_teams(xml_document, home_team, away_team, card_type='y'):
    assert card_type == 'y' or card_type == 'r', "Please enter either y or r"
    tree = ET.fromstring(xml_document)
    stat_home_team = 0
    stat_away_team = 0
    
    #Dealing with card type using the root element & the card type argument
    if tree.tag == 'card':
        for child in tree.iter('value'):
            #Some xml docs have no card_type element in the tree. comment section seems to have that information
            try:
                if child.find('comment').text == card_type:
                    if int(child.find('team').text) == home_team:
                        stat_home_team += 1
                    else:
                        stat_away_team += 1
            except AttributeError:
                #Some values in the xml doc don't have team values, so there isn't much we can do at this stage
                pass
                
        return stat_home_team, stat_away_team
    
    #Lets take the last possession stat which is available from the xml doc
    if tree.tag == 'possession':
        try:
            last_value = [child for child in tree.iter('value')][-1]
            return int(last_value.find('homepos').text), int(last_value.find('awaypos').text)
        except:
            return None, None
    
    #Taking care of all other stats by extracting based on the home team & away team api id's
    for team in [int(stat.text) for stat in tree.findall('value/team')]:
        if team == home_team: 
            stat_home_team += 1
        else:
            stat_away_team += 1
    return stat_home_team, stat_away_team

In [12]:
epl[['on_target_shot_home_team','on_target_shot_away_team']] = epl[['shoton','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['shoton'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")  
epl[['off_target_shot_home_team','off_target_shot_away_team']] = epl[['shotoff','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['shotoff'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand") 
epl[['foul_home_team','foul_away_team']] = epl[['foulcommit','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['foulcommit'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")  
epl[['yellow_card_home_team','yellow_card_away_team']] = epl[['card','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['card'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")
epl[['red_card_home_team','red_card_away_team']] = epl[['card','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['card'],x['home_team_api_id'],x['away_team_api_id'], card_type='r'), axis = 1,result_type="expand")  
epl[['crosses_home_team','crosses_away_team']] = epl[['cross','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['cross'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")  
epl[['corner_home_team','corner_away_team']] = epl[['corner','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['corner'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")  
epl[['possession_home_team','possession_away_team']] = epl[['possession','home_team_api_id','away_team_api_id']].apply(lambda x: calculate_stats_both_teams(x['possession'],x['home_team_api_id'],x['away_team_api_id']), axis = 1,result_type="expand")

In [13]:
epl.describe()

,id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,...,yellow_card_home_team,yellow_card_away_team,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team
count,3040.000000,3040.0,3040.00000,3.040000e+03,3040.00000,3040.00000,3040.000000,3040.000000,3040.000000,3040.000000,...,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3034.000000,3034.000000
mean,3248.500000,1729.0,19.50000,1.178767e+06,9142.27500,9142.27500,1.550987,1.159539,2.701964,3.952720,...,1.432566,1.819737,0.041776,0.052961,20.917763,16.252632,6.126645,4.811513,51.676994,48.323006
std,877.716735,0.0,10.96766,4.909627e+05,753.08936,753.08936,1.311615,1.144629,1.689834,0.998305,...,1.171699,1.293568,0.204984,0.232639,8.736225,7.376625,3.147689,2.718382,8.640939,8.640939
min,1729.000000,1729.0,1.00000,4.890420e+05,8191.00000,8191.00000,0.000000,0.000000,1.100000,3.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,4.000000,20.000000
25%,2488.750000,1729.0,10.00000,7.946915e+05,8528.00000,8528.00000,1.000000,0.000000,1.670000,3.300000,...,1.000000,1.000000,0.000000,0.000000,15.000000,11.000000,4.000000,3.000000,46.000000,43.000000
50%,3248.500000,1729.0,19.50000,1.127220e+06,8659.00000,8659.00000,1.000000,1.000000,2.200000,3.600000,...,1.000000,2.000000,0.000000,0.000000,20.000000,15.000000,6.000000,4.000000,52.000000,48.000000
75%,4008.250000,1729.0,29.00000,1.537336e+06,9879.00000,9879.00000,2.000000,2.000000,3.000000,4.200000,...,2.000000,3.000000,0.000000,0.000000,26.000000,21.000000,8.000000,6.000000,57.000000,54.000000
max,4768.000000,1729.0,38.00000,1.989079e+06,10261.00000,10261.00000,9.000000,6.000000,15.000000,11.000000,...,7.000000,9.000000,2.000000,2.000000,72.000000,55.000000,20.000000,19.000000,80.000000,96.000000


In [14]:
epl.describe()

,id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,B365H,B365D,...,yellow_card_home_team,yellow_card_away_team,red_card_home_team,red_card_away_team,crosses_home_team,crosses_away_team,corner_home_team,corner_away_team,possession_home_team,possession_away_team
count,3040.000000,3040.0,3040.00000,3.040000e+03,3040.00000,3040.00000,3040.000000,3040.000000,3040.000000,3040.000000,...,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3040.000000,3034.000000,3034.000000
mean,3248.500000,1729.0,19.50000,1.178767e+06,9142.27500,9142.27500,1.550987,1.159539,2.701964,3.952720,...,1.432566,1.819737,0.041776,0.052961,20.917763,16.252632,6.126645,4.811513,51.676994,48.323006
std,877.716735,0.0,10.96766,4.909627e+05,753.08936,753.08936,1.311615,1.144629,1.689834,0.998305,...,1.171699,1.293568,0.204984,0.232639,8.736225,7.376625,3.147689,2.718382,8.640939,8.640939
min,1729.000000,1729.0,1.00000,4.890420e+05,8191.00000,8191.00000,0.000000,0.000000,1.100000,3.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,4.000000,20.000000
25%,2488.750000,1729.0,10.00000,7.946915e+05,8528.00000,8528.00000,1.000000,0.000000,1.670000,3.300000,...,1.000000,1.000000,0.000000,0.000000,15.000000,11.000000,4.000000,3.000000,46.000000,43.000000
50%,3248.500000,1729.0,19.50000,1.127220e+06,8659.00000,8659.00000,1.000000,1.000000,2.200000,3.600000,...,1.000000,2.000000,0.000000,0.000000,20.000000,15.000000,6.000000,4.000000,52.000000,48.000000
75%,4008.250000,1729.0,29.00000,1.537336e+06,9879.00000,9879.00000,2.000000,2.000000,3.000000,4.200000,...,2.000000,3.000000,0.000000,0.000000,26.000000,21.000000,8.000000,6.000000,57.000000,54.000000
max,4768.000000,1729.0,38.00000,1.989079e+06,10261.00000,10261.00000,9.000000,6.000000,15.000000,11.000000,...,7.000000,9.000000,2.000000,2.000000,72.000000,55.000000,20.000000,19.000000,80.000000,96.000000


In [13]:
epl.loc[epl['possession_home_team'].isnull()]
##There is only 6 games whose possession stats were not available which represent only a few pourcentage of the data therefore we can drop them
epl=epl.drop(epl.loc[epl['possession_home_team'].isnull()].index, inplace=True)


In [15]:
epl.describe()

AttributeError: 'NoneType' object has no attribute 'describe'

In [13]:
epl.columns

AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
##drop the columns that are not required
df= epl.loc[:,'season':'possession_away_team']

In [130]:
#drop column goal
df=df.drop(['goal','B365H','B365D','B365A'], axis=1, inplace=True)
##To reduce the number of columns since we dont want to overfitt and we want to predict to the result for the Home team we can create difference vectors for each features
## Moreover I think that yellow card isn't a good feature however we need to take care of the red cards that can change completly the outcome of the match
##For each row in the dataframe we can create a vector of the difference between the home team and away team stats
## Plus tard je pourais rajouter les stats des équipes pour linstant nike


AttributeError: 'NoneType' object has no attribute 'drop'

In [101]:
def differencevector(row):
    """
    This function is applied to each row in the 'matches' DataFrame.
    For each row, it returns a pandas.Series containing the number of
    goals scored by each team and the result from the home team's
    perspective.
    
    Keyword arguments:
    row -- pandas.Series
    """
    homegoal = row['home_team_goal'] 
    awaygoal = row['away_team_goal']
    
   
    if (homegoal > awaygoal):
        row.at['result']='2'
        print(row['result'])

    elif (awaygoal == homegoal):
        row['result'] = 1
    else:
        row['result'] = 0
    

        

In [104]:
##add column result with no value with the result for the home team 2=win, 1=draw, 0=loss
df.loc[df['home_team_goal'] > df['away_team_goal'], 'result'] = 2
df.loc[df['home_team_goal'] ==df['away_team_goal'], 'result'] = 1
df.loc[df['home_team_goal'] < df['away_team_goal'], 'result'] = 0

In [113]:
df['card']= df['red_card_away_team'] - df['red_card_home_team'] 
df['cross']=df['crosses_home_team'] - df['crosses_away_team']
df['corner']=df['corner_home_team'] - df['corner_away_team']
df['possession']=df['possession_home_team']

In [114]:
df=df.drop(['foulcommit','yellow_card_away_team','red_card_home_team','red_card_away_team','crosses_home_team','crosses_away_team','corner_home_team','corner_away_team','possession_home_team','possession_away_team'], axis=1, inplace=True)

In [110]:
df.columns

Index(['season', 'date', 'match_api_id', 'home_team_api_id', 'home_team',
       'away_team_api_id', 'away_team', 'home_team_goal', 'away_team_goal',
       'foulcommit', 'card', 'cross', 'corner', 'possession',
       'on_target_shot_home_team', 'on_target_shot_away_team',
       'off_target_shot_home_team', 'off_target_shot_away_team',
       'foul_home_team', 'foul_away_team', 'yellow_card_home_team',
       'yellow_card_away_team', 'red_card_home_team', 'red_card_away_team',
       'crosses_home_team', 'crosses_away_team', 'corner_home_team',
       'corner_away_team', 'possession_home_team', 'possession_away_team',
       'result'],
      dtype='object')